In [ ]:
# ============ Base imports ======================
import os
import shlex
import subprocess as sp
from functools import partial
# ====== External package imports ================
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
# ====== Internal package imports ================
from src.modules.data.database_io import DatabaseIO
# ============== Logging  ========================
import logging
from src.modules.utils.setup import setup, IndentLogger
logger = IndentLogger(logging.getLogger(''), {})
# =========== Config File Loading ================
from src.modules.utils.config_loader import get_config
conf  = get_config()
# ================================================

In [2]:
dbio = DatabaseIO()

In [3]:
pluit1, columns = dbio.get_results_motion('.mkv',93)

In [4]:
pluit1_df = pd.DataFrame(list(pluit1), columns=columns)

In [ ]:
pluit1_df.head()

In [6]:
pluit1_df['xc'] = (pluit1_df['xtl']+pluit1_df['xbr'])/2
pluit1_df['yc'] = (pluit1_df['ytl']+pluit1_df['ybr'])/2

what region do I care about?

In [ ]:
height = 1080
width = 1920
frps = 5
path = '.mkv'
imsize = 3 * height * width  # 3 bytes per pixel
print(f"Reading from file: {path}")
commands = shlex.split(f'ffmpeg -i {path} -vf fps=1 -ss 250 -to 350 -f image2pipe -pix_fmt rgb24 -vcodec rawvideo -')
p = sp.Popen(commands, stdout=sp.PIPE, stderr=sp.DEVNULL, bufsize=int(imsize))
i = 0
frame_list = []
for raw_frame in iter(partial(p.stdout.read, imsize), ''):
    i += 1
    try:
        frame = np.fromstring(raw_frame, dtype='uint8').reshape((height, width, 3))
        frame_list.append(frame)
    except Exception as e:
        print(f"Done reading from file: {path}")
        break

In [12]:
def show_img(image):
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.axis('off')
    plt.imshow(image)

In [ ]:
show_img(frame_list[0])
frame=frame_list[0]

Draw region on image in order to test out what is the right polygon

In [ ]:
frame = frame_list[1].copy()
tl = (800,200)
tr = (960,200)
bl = (326,508)
br = (533,529)
cv2.line(frame, tl, tr, (255,0,0), 2)
cv2.line(frame, tr, br, (255,0,0), 2)
cv2.line(frame, br, bl, (255,0,0), 2)
cv2.line(frame, bl, tl, (255,0,0), 2)
cv2.line(frame, (430,510),(880,200),(0,255,0),2)
show_img(frame)

In [8]:
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    from scipy.spatial import Delaunay
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

Define which objects are in that region

In [15]:
pluit_coord = [(pluit1_df.loc[ii,'xc'],pluit1_df.loc[ii,'ybr']) for ii in range(len(pluit1_df))]

In [23]:
pluit1_in_area = pluit1_df.loc[in_hull(pluit_coord, [tl,tr,br,bl])]

In [18]:
direction = np.array([880-430,200-510])
direction = direction/np.linalg.norm(direction)

In [ ]:
direction

In [ ]:
pluit1_in_area.loc[:,'magnitude'] = np.sqrt(pluit1_in_area['mean_delta_x']**2 + pluit1_in_area['mean_delta_y']**2)

In [27]:
pluit1_df3 = pluit1_in_area[pluit1_in_area['magnitude']>0]

In [ ]:
len(pluit1_df3)

In [ ]:
pluit1_df3['cos'] = (direction[0]*pluit1_df3['mean_delta_x']+direction[1]*pluit1_df3['mean_delta_y'])/pluit1_df3['magnitude']

In [ ]:
plt.hist(pluit1_df3['cos'], bins=20)
plt.show()

Find objects in that image coming down the wrong direction

In [ ]:
pluit1_wrongdir = pluit1_df3[pluit1_df3['cos']<-.8]

In [ ]:
len(pluit1_wrongdir)

In [ ]:
plt.hist(pluit1_wrongdir['frame_number'],bins=40)

In [ ]:
labels_we_test = ['car','truck','bicycle','motorbike','pedestrian','bus']
pluit1_wrongdir['label'] = pluit1_wrongdir[labels_we_test].idxmax(axis=1)
pluit1_wrongdir['confidence'] = pluit1_wrongdir[labels_we_test].max(axis=1)

Here, select for cars detected with confidence above 0.4

In [53]:
pluit1_wrongdir = pluit1_wrongdir[(pluit1_wrongdir['label']=='car') & (pluit1_wrongdir['confidence']>.4)]

In [72]:
pluit1_wrongdir = pluit1_wrongdir[pluit1_wrongdir['magnitude']>10].sort_values('frame_number')

In [ ]:
pluit1_wrongdir

In [47]:
v = VideoFile(f'{conf.dirs.videos}.mkv')

Check a video segment where we might have found something...

In [ ]:
clip = v.video_file_clip.subclip(3357.5,3363)
#clip = clip.fl_time(lambda t: t/25.)
#clip = clip.set_duration(50.)
mpy.ipython_display(clip, width=500, maxduration=100,fps=6)

Save it!

In [ ]:
clip.write_videofile('clips_0813/events1.mkv',
                                        fps=6,codec='h264')

In [ ]:
clip = v.video_file_clip.subclip(3445,3455)
#clip = clip.fl_time(lambda t: t/25.)
#clip = clip.set_duration(50.)
mpy.ipython_display(clip, width=500, maxduration=100)

We want to find close-by frames where we detect such objects, to ensure the detection is not spurious. This plot helps with that:

In [ ]:
plt.plot(np.array(pluit1_wrongdir['frame_number'].rolling(5).std()))
plt.yscale('log')

### Finally, let's make a function carrying out all these calculations, in order to apply this method to more videos:

In [112]:
def plot_rolling_std_wrong_direction_cars(vid_ref,confidence=.4,class_of_interest='car'):
    pluit2, columns = dbio.get_results_motion(vid_ref,93)
    pluit2_df = pd.DataFrame(list(pluit2), columns=columns)
    pluit2_df['xc'] = (pluit2_df['xtl']+pluit2_df['xbr'])/2
    pluit2_coord = [(pluit2_df.loc[ii,'xc'],pluit2_df.loc[ii,'ybr']) for ii in range(len(pluit2_df))]
    pluit2_in_area = pluit2_df.loc[in_hull(pluit2_coord, [tl,tr,br,bl])]
    pluit2_in_area.loc[:,'magnitude'] = np.sqrt(pluit2_in_area['mean_delta_x']**2 + pluit2_in_area['mean_delta_y']**2)
    pluit2_df3 = pluit2_in_area[pluit2_in_area['magnitude']>0]
    pluit2_df3['cos'] = (direction[0]*pluit2_df3['mean_delta_x']+direction[1]*pluit2_df3['mean_delta_y'])/pluit2_df3['magnitude']
    pluit2_wrongdir = pluit2_df3[pluit2_df3['cos']<-.8]
    pluit2_wrongdir['label'] = pluit2_wrongdir[labels_we_test].idxmax(axis=1)
    pluit2_wrongdir['confidence'] = pluit2_wrongdir[labels_we_test].max(axis=1)
    pluit2_wrongdir = pluit2_wrongdir[(pluit2_wrongdir['label']=='car') & (pluit2_wrongdir['confidence']>.4)]
    pluit2_wrongdir = pluit2_wrongdir[pluit2_wrongdir['magnitude']>10].sort_values('frame_number')
    plt.plot(np.array(pluit2_wrongdir['frame_number'].rolling(5).std()), 'bo', np.array(pluit2_wrongdir['frame_number'].rolling(5).std()), 'k')
    plt.yscale('log')
    plt.show()
    return pluit2_wrongdir

In [ ]:
wrongdir_df = plot_rolling_std_wrong_direction_cars('.mkv')

In [ ]:
wrongdir_df = plot_rolling_std_wrong_direction_cars('.mkv')

When we find a low spot in the graphs, this tells us there is such an event, and we can go look at the video...

In [ ]:
wrongdir_df.iloc[35:]

In [ ]:
v5 = VideoFile('.mkv')

In [ ]:
clip = v5.video_file_clip.subclip(3686,3696)
#clip = clip.fl_time(lambda t: t/25.)
#clip = clip.set_duration(50.)
mpy.ipython_display(clip, width=500, maxduration=100, fps=6)

In [ ]:
wrongdir_df = plot_rolling_std_wrong_direction_cars('.mkv')

In [ ]:
wrongdir_df.iloc[19:30]

In [ ]:
v2 = VideoFile('.mkv')

In [ ]:
clip = v2.video_file_clip.subclip(3830,3840)
mpy.ipython_display(clip, width=500, maxduration=100, fps=6)

In [ ]:
clip.write_videofile('clips_0813/events2.mkv',
                                        fps=6,codec='h264')

### Note: cosine cut at 0.95 instead of 0.8 seems to work well for all events we detected